<a href="https://colab.research.google.com/github/gox6/buzzer/blob/main/buzzer_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data cleanup

In [ ]:
!pip install optuna

In [4]:
import pandas as pd
import polars as pl

In [5]:
# Getting data from Google Drive file
url='https://drive.google.com/uc?id=1FrSdVAw_nmouSCE7VZ54-_HfM_C9DIC6'
df_orig = pl.read_csv(url)

In [6]:
# Renaming columns using snake_case
df = df_orig[:]
to_snake_case = lambda x: '_'.join(x.lower().split(' '))
df.columns = list(map(to_snake_case, df_orig.columns))

In [7]:
df.head(5)

lead_number,lead_origin,lead_source,do_not_email,do_not_call,converted,totalvisits,total_time_spent_on_website,page_views_per_visit,last_activity,country,specialization,how_did_you_hear_about_x_education,what_is_your_current_occupation,what_matters_most_to_you_in_choosing_a_course,search,magazine,newspaper_article,x_education_forums,newspaper,digital_advertisement,through_recommendations,receive_more_updates_about_our_courses,tags,lead_quality,update_me_on_supply_chain_content,get_updates_on_dm_content,lead_profile,city,asymmetrique_activity_index,asymmetrique_profile_index,asymmetrique_activity_score,asymmetrique_profile_score,i_agree_to_pay_the_amount_through_cheque,a_free_copy_of_mastering_the_interview,last_notable_activity
i64,str,str,str,str,i64,f64,i64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str
660737,"""API""","""Olark Chat""","""No""","""No""",0,0.0,0,0.0,"""Page Visited o…","""India""","""Others""",null,"""Unemployed""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Interested in …","""Low in Relevan…","""No""","""No""",null,null,"""02.Medium""","""02.Medium""",15.0,15.0,"""No""","""No""","""Modified"""
660728,"""API""","""Organic Search…","""No""","""No""",0,5.0,674,2.5,"""Email Opened""","""India""","""Others""",null,"""Unemployed""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Ringing""","""Not Sure""","""No""","""No""",null,null,"""02.Medium""","""02.Medium""",15.0,15.0,"""No""","""No""","""Email Opened"""
660727,"""Landing Page S…","""Direct Traffic…","""No""","""No""",1,2.0,1532,2.0,"""Email Opened""","""India""","""Business Admin…",null,"""Student""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Will revert af…","""Might be""","""No""","""No""","""Potential Lead…","""Mumbai""","""02.Medium""","""01.High""",14.0,20.0,"""No""","""Yes""","""Email Opened"""
660719,"""Landing Page S…","""Direct Traffic…","""No""","""No""",0,1.0,305,1.0,"""Unreachable""","""India""","""Media and Adve…","""Word Of Mouth""","""Unemployed""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Ringing""","""Not Sure""","""No""","""No""",null,"""Mumbai""","""02.Medium""","""01.High""",13.0,17.0,"""No""","""No""","""Modified"""
660681,"""Landing Page S…","""Google""","""No""","""No""",1,2.0,1428,1.0,"""Converted to L…","""India""","""Others""","""Other""","""Unemployed""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Will revert af…","""Might be""","""No""","""No""",null,"""Mumbai""","""02.Medium""","""01.High""",15.0,18.0,"""No""","""No""","""Modified"""


In [8]:
df.describe()

describe,lead_number,lead_origin,lead_source,do_not_email,do_not_call,converted,totalvisits,total_time_spent_on_website,page_views_per_visit,last_activity,country,specialization,how_did_you_hear_about_x_education,what_is_your_current_occupation,what_matters_most_to_you_in_choosing_a_course,search,magazine,newspaper_article,x_education_forums,newspaper,digital_advertisement,through_recommendations,receive_more_updates_about_our_courses,tags,lead_quality,update_me_on_supply_chain_content,get_updates_on_dm_content,lead_profile,city,asymmetrique_activity_index,asymmetrique_profile_index,asymmetrique_activity_score,asymmetrique_profile_score,i_agree_to_pay_the_amount_through_cheque,a_free_copy_of_mastering_the_interview,last_notable_activity
str,f64,str,str,str,str,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str
"""count""",9240.0,"""9240""","""9240""","""9240""","""9240""",9240.0,9240.0,9240.0,9240.0,"""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""","""9240""",9240.0,9240.0,"""9240""","""9240""","""9240"""
"""null_count""",0.0,"""0""","""36""","""0""","""0""",0.0,137.0,0.0,137.0,"""103""","""0""","""0""","""7250""","""0""","""2709""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""6855""","""3669""","""4218""","""4218""",4218.0,4218.0,"""0""","""0""","""0"""
"""mean""",617188.435606,null,null,null,null,0.38539,3.445238,487.698268,2.36282,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,14.306252,16.344883,null,null,null
"""std""",23405.995698,null,null,null,null,0.486714,4.854853,548.021466,2.161418,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.386694,1.811395,null,null,null
"""min""",579533.0,"""API""","""Click2call""","""No""","""No""",0.0,0.0,0.0,0.0,"""Approached upf…","""Asia/Pacific R…","""Banking, Inves…","""Advertisements…","""Businessman""","""Better Career …","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Already a stud…","""High in Releva…","""No""","""No""","""Dual Specializ…","""Mumbai""","""01.High""","""01.High""",7.0,11.0,"""No""","""No""","""Approached upf…"
"""max""",660737.0,"""Quick Add Form…","""youtubechannel…","""Yes""","""Yes""",1.0,251.0,2272.0,55.0,"""Visited Booth …","""unknown""","""Travel and Tou…","""Word Of Mouth""","""Working Profes…","""Other""","""Yes""","""No""","""Yes""","""Yes""","""Yes""","""Yes""","""Yes""","""No""","""wrong number g…","""Worst""","""No""","""No""","""Student of Som…","""Tier II Cities…","""03.Low""","""03.Low""",18.0,20.0,"""No""","""Yes""","""View in browse…"
"""median""",615479.0,null,null,null,null,0.0,3.0,248.0,2.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,14.0,16.0,null,null,null
"""25%""",596485.0,null,null,null,null,0.0,1.0,12.0,1.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,14.0,15.0,null,null,null
"""75%""",637388.0,null,null,null,null,1.0,5.0,936.0,3.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,15.0,18.0,null,null,null


In [9]:
# Types of variables

for column, type_ in zip(df.columns, df.dtypes):
  print(f"{column:<50}{type_}")

lead_number                                       Int64
lead_origin                                       Utf8
lead_source                                       Utf8
do_not_email                                      Utf8
do_not_call                                       Utf8
converted                                         Int64
totalvisits                                       Float64
total_time_spent_on_website                       Int64
page_views_per_visit                              Float64
last_activity                                     Utf8
country                                           Utf8
specialization                                    Utf8
how_did_you_hear_about_x_education                Utf8
what_is_your_current_occupation                   Utf8
what_matters_most_to_you_in_choosing_a_course     Utf8
search                                            Utf8
magazine                                          Utf8
newspaper_article                                 Utf8
x

In [10]:
# Assigning roles to variables

df = df.with_columns(pl.col('total_time_spent_on_website').cast(pl.Float64))

cat_vars = df.select([pl.col(pl.Utf8)]).columns
num_vars = df.select([pl.col(pl.Float64)]).columns
skip_vars = ['lead_number']
target = 'converted'

# Check that all variables were assigned the role
assert len(df.columns) == len(cat_vars + num_vars + skip_vars + [target])


In [11]:
from sklearn.model_selection import train_test_split

X = df.select(cat_vars + num_vars)
y = df.select(pl.col(target)).to_numpy().reshape(-1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

In [17]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from functools import reduce
import numpy as np

cat_enc = OrdinalEncoder()
cat_enc.fit(X.select(cat_vars))

std_scaler =  StandardScaler()
std_scaler.fit(X_train.select(num_vars))

def preprocess(X, encoders):
    X_partial_encodings = map(lambda enc: enc.transform(X.select(enc.feature_names_in_)), encoders)
    return reduce(lambda x, y: np.concatenate([x, y], axis=1), X_partial_encodings)

X_train_pp = preprocess(X_train, encoders=[cat_enc, std_scaler])
X_test_pp = preprocess(X_test, encoders=[cat_enc, std_scaler])

In [149]:
import lightgbm as lgb 

params = {'objective': 'binary', 'n_estimators': 5000}

def train_lgb(X_train, y_train,
              X_test, y_test,
              X_val=None, y_val=None,
              params = {'objective': 'binary', 'n_estimators': 5000},
              ):
  
    std_scaler =  StandardScaler()
    std_scaler.fit(X_train.select(num_vars))

    X_train_pp = preprocess(X_train, encoders=[cat_enc, std_scaler])
    if X_val is not None:
        X_val_pp = preprocess(X_val, encoders=[cat_enc, std_scaler])
    X_test_pp = preprocess(X_test, encoders=[cat_enc, std_scaler])
    
    clf = lgb.sklearn.LGBMClassifier(**params)

    early_stopping_patience = 100 if X_val is not None else 5000
    callbacks = [lgb.early_stopping(early_stopping_patience, verbose=False), lgb.log_evaluation(period=0)]
    eval_set = [(X_val_pp, y_val)] #if X_val is not None else [(X_train_pp, y_train)]

    clf.fit(X_train_pp, y_train,
            eval_set =  eval_set,
            eval_names = 'val' if X_val is not None else 'train',
            callbacks=callbacks,
            feature_name = cat_vars + num_vars,
            #categorical_feature=list(range(len(cat_vars))),
            )

    return clf.score(X_test_pp, y_test)

In [136]:
from sklearn.model_selection import KFold
from statistics import mean

def cross_validate(X_train, y_train, n_folds=5, params={'objective': 'binary', 'n_estimators': 5000}):

    kf5 = KFold(n_splits=n_folds, shuffle=True, random_state=0)

    metrics = []

    for train_train_indices, train_val_test_indices in kf5.split(X_train):

        X_train_train = X_train[train_train_indices]
        y_train_train = y_train[train_train_indices]

        X_train_val_test = X_train[train_val_test_indices]
        y_train_val_test = y_train[train_val_test_indices]
        
        X_train_val, X_train_test, y_train_val, y_train_test = train_test_split(X_train_val_test, y_train_val_test, test_size=0.5, random_state=0)

        metric = train_lgb(X_train_train, y_train_train,
                           X_train_test, y_train_test,
                           X_val=X_train_val, y_val=y_train_val,
                           params = params)
        
        metrics.append(metric)

    return mean(metrics)
        
  

In [150]:
def objective(trial):
    # Define hyperparameters
    params = {
        'objective': 'binary',
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-2),
        'num_leaves': trial.suggest_int('num_leaves', 2, 128),
        'scale_pos_weight': trial.suggest_int('scale_pos_weight', 1, 10),
        'n_estimators': 5000,
    }
    
    # Train model
    metric = cross_validate(X_train, y_train, n_folds=5, params=params)
    
    # Return cross-validated accuracy
    return metric

import optuna
study = optuna.create_study(direction='maximize', study_name="Hyperparameters tuning")
study.optimize(objective, n_trials=10)

[I 2023-05-08 01:35:49,799] A new study created in memory with name: Hyperparameters tuning
[I 2023-05-08 01:36:13,612] Trial 0 finished with value: 0.9210148849797023 and parameters: {'learning_rate': 0.004025563833374624, 'num_leaves': 6, 'scale_pos_weight': 5}. Best is trial 0 with value: 0.9210148849797023.
[W 2023-05-08 01:36:58,866] Trial 1 failed with parameters: {'learning_rate': 0.000880372841871243, 'num_leaves': 22, 'scale_pos_weight': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-150-12eb9e17c126>", line 12, in objective
    metric = cross_validate(X_train, y_train, n_folds=5, params=params)
  File "<ipython-input-136-83959ba052d5>", line 20, in cross_validate
    metric = train_lgb(X_train_train, y_train_train,
  File "<ipython-input-149-edb80afedaf5>", line 25, in train_lgb

KeyboardInterrupt: ignored

In [141]:
best_params = study.best_params
best_params

{'learning_rate': 0.004858456696051978,
 'num_leaves': 86,
 'scale_pos_weight': 1}